In [1]:
import pandas as pd

In [2]:
charts = pd.read_csv('### File Path ###')

In [ ]:
charts = charts.rename(columns={"Unnamed: 0": "rank"})

In [ ]:
# adding value counts column to show frequency of each song's occurances
uri_occurances = charts['uri'].value_counts()
uri_occurances_dic = uri_occurances.to_dict()
charts['occurances'] = charts['uri'].map(uri_occurances_dic) 

In [ ]:
# subsetting down to only the relevent data for this social network graph
charts = charts[['country', 'uri']]

# Creating a nodes list (for social network analysis)

In [ ]:
country_names = charts.country.unique()

In [8]:
#creating df
nodes = pd.DataFrame(country_names)
#adding id column
nodes.insert(0, 'country_id', range(1, 1 + len(nodes)))
#renaming columns
nodes.columns = ['id', 'label']

NameError: name 'country_names' is not defined

In [ ]:
nodes.to_csv('### Nodes File Path ###')

# Creating an edges list (for social network analysis)

In [ ]:
# joining country ids from nodes list onto edges list
inner_join = pd.merge(charts,  
                      nodes,  
                      on ='country',  
                      how ='left')

In [ ]:
# dictionary of country ids and their respective list of uris
uris_by_id = {}
for id in list(range(1,63)):
    country_table = inner_join[inner_join['country_id'] == id]
    country_uri_list = list(country_table['uri'])
    uris_by_id[id] = country_uri_list

In [ ]:
# appends a tuple (country1, country2) for every song country1 and country2 have in common
in_common_list = []
for id in uris_by_id: #loops thru every country
    for uri in uris_by_id[id]: #loops thru every uri in that country

        for key in uris_by_id:   #loops thru other countries
            if key == id:         #skips itself
                continue
            if uri in uris_by_id[key]:     #loops thru uris
                in_common_list.append((id,key))
                print(uri, id, key)

In [ ]:
# counts the occurances of a particular tuple in tuple list
# determines the 'weight' of a particular edge -- proprortional to number of songs country1 and country2 have in common
tuple_dic = {}
for item in in_common_list:
    if item in tuple_dic:
        tuple_dic[item] += 1
    else:
        tuple_dic[item] = 1

In [ ]:
#check for mistakes -- making sure countries don't have more than 50 songs in common
for key in tuple_dic:
    if tuple_dic[key] > 50:
        print("oh no, more than 50 songs in common")

In [ ]:
# create edges dataframe
edges = pd.DataFrame.from_dict(tuple_dic, orient = 'index')

edges.columns = ['weight']
edges['tuple'] = edges.index

edges['source'] = edges.apply(lambda x: x[1][0], axis=1)
edges['target'] = edges.apply(lambda x: x[1][1], axis=1)
edges['type'] = 'Undirected'

In [ ]:
# reorder columns for Gephi
edges = edges[['source','target','type','weight']]
edges.reset_index()
edges.to_csv('### Filepath ###')

# next, import node list and edge list into Gephi

In [ ]:
# Gephi settings used:
# layout = ForceAtlas
# color based on modularity class
# text size proportional to degree